# **Import Modules**

## *python*

In [1]:
# Import python modules
import os,sys

## *custom*

In [2]:
# Import customised modules

# Define path
sys.path.append(os.path.abspath(os.path.join('..')))

##import relevant modules
try:
    from scripts._02_2_embed_and_index import EmbeddingIndexer

except ImportError:
    print('Could not import module from `scripts/`.')

c:\Users\nuhamin\Documents\kifiya\week 6\Intelligent-Complaint-Analysis-for-Financial-Services\.chatvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Embedding and Indexing**

In [3]:
# Define DataFrame path
df_chunks_path = ('../data/processed/chunked_complaints.csv')
vector_store_dir = ('../vector store')

In [4]:
# Initalise class
embedder_and_indexer = EmbeddingIndexer (df_chunks_path = df_chunks_path, vector_store_dir = vector_store_dir)

In [5]:
# Load data
embedder_and_indexer.load_chunks()

DataFrame loaded successfully from ..\data\processed\chunked_complaints.csv


,Chunk,Chunk Length,Company,Company Public Response,Company Response To Consumer,Complaint Id,Complaint Length,Consumer Complaint Narrative,Consumer Complaint Narrative Clean,Consumer Consent Provided?,...,Date Sent To Company,Issue,Product,State,Sub-Issue,Sub-Product,Submitted Via,Tags,Timely Response?,Zip Code
0,xxxx xxxx card opened name fraudster. received...,311,"CITIBANK, N.A.",Company has responded to the consumer and the ...,Closed with non-monetary relief,14069121,488.0,A XXXX XXXX card was opened under my name by a...,xxxx xxxx card opened name fraudster. received...,Consent provided,...,2025-06-13,Getting a credit card,Credit card,TX,Card opened without my consent or knowledge,Store credit card,Web,Servicemember,Yes,78230
1,made mistake using wellsfargo debit card depsi...,354,WELLS FARGO & COMPANY,Company has responded to the consumer and the ...,Closed with explanation,14061897,555.0,I made the mistake of using my wellsfargo debi...,made mistake using wellsfargo debit card depsi...,Consent provided,...,2025-06-13,Managing an account,Checking or savings account,ID,Deposits and withdrawals,Checking account,Web,Unknown,Yes,83815
2,secured credit card citibank changed unsecured...,496,"CITIBANK, N.A.",Company has responded to the consumer and the ...,Closed with monetary relief,14047085,806.0,"Dear CFPB, I have a secured credit card with c...",secured credit card citibank changed unsecured...,Consent provided,...,2025-06-13,"Other features, terms, or problems",Credit card,NY,Other problem,General-purpose credit card or charge card,Web,Unknown,Yes,11220
3,citi reward cards. credit balance issued $8400...,443,"CITIBANK, N.A.",Company has responded to the consumer and the ...,Closed with explanation,14040217,1199.0,I have a Citi rewards cards. The credit balanc...,citi reward cards. credit balance issued $8400...,Consent provided,...,2025-06-12,Incorrect information on your report,Credit card,IL,Account information incorrect,General-purpose credit card or charge card,Web,Unknown,Yes,60067
4,.00 $5400.00 sends message reached credit limi...,364,"CITIBANK, N.A.",Company has responded to the consumer and the ...,Closed with explanation,14040217,1199.0,I have a Citi rewards cards. The credit balanc...,citi reward cards. credit balance issued $8400...,Consent provided,...,2025-06-12,Incorrect information on your report,Credit card,IL,Account information incorrect,General-purpose credit card or charge card,Web,Unknown,Yes,60067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022259,". resolution outside court would preferred, ma...",329,WELLS FARGO & COMPANY,Company has responded to the consumer and the ...,Closed with explanation,6238646,3462.0,"I already have a case XXXX, but it should not ...","already case xxxx, closed. additional informat...",Consent provided,...,2022-11-23,Managing an account,Checking or savings account,OK,Deposits and withdrawals,Savings account,Web,Unknown,Yes,73160
1022260,xxxxxxxx final payment supposed submitted want...,214,"Westlake Services, LLC",Company has responded to the consumer and the ...,Closed with explanation,6238123,2284.0,On XX/XX/XXXX my final payment was supposed to...,xxxxxxxx final payment supposed submitted want...,Consent provided,...,2022-11-23,Vehicle was repossessed or sold the vehicle,"Payday loan, title loan, or personal loan",CA,Unknown,Title loan,Web,Older American,Yes,90059
1022261,. called immediately gave represented new addr...,429,"Westlake Services, LLC",Company has responded to the consumer and the ...,Closed with explanation,6238123,2284.0,On XX/XX/XXXX my final payment was supposed to...,xxxxxxxx final payment supposed submitted want...,Consent provided,...,2022-11-23,Vehicle was repossessed or sold the vehicle,"Payday loan, title loan, or personal loan",CA,Unknown,Title loan,Web,Older American,Yes,90059
1022262,.00 going balance rest going interest fee see ...,493,"Westlake Services, LLC",Company has responded to the consu

In [ ]:
# Embed and Index chunks
embedder_and_indexer.index_chunks()

Indexing chunks into ChromaDB...


  6%|▌         | 63106/1022264 [2:08:13<21:11:07, 12.58it/s]     